## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-16-42-22 +0000,bbc,Unite votes to suspend Rayner's membership ove...,https://www.bbc.com/news/articles/cx24de0d9rdo
1,2025-07-11-16-41-00 +0000,wsj,Canada Unemployment Rate Slips to 6.9% in June,https://www.wsj.com/economy/central-banking/ca...
2,2025-07-11-16-33-10 +0000,nypost,Hero Camp Mystic director who died trying to s...,https://nypost.com/2025/07/11/us-news/hero-tex...
3,2025-07-11-16-29-33 +0000,nyt,Germany Takes an Unassuming Approach to Tax Cu...,https://www.nytimes.com/2025/07/11/world/europ...
4,2025-07-11-16-15-37 +0000,nypost,Pennsylvania man who posted video of father’s ...,https://nypost.com/2025/07/11/us-news/pennsylv...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2328/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
32,trump,16
91,will,5
383,brazil,5
169,where,4
133,tariff,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
19,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,43
10,2025-07-11-15-59-15 +0000,nypost,Head Start education program will be cut off f...,https://nypost.com/2025/07/11/us-news/head-sta...,39
50,2025-07-11-11-21-26 +0000,nyt,Behind Trump’s Decision to Tax Brazil to Save ...,https://www.nytimes.com/2025/07/11/world/ameri...,33
88,2025-07-10-20-10-28 +0000,nyt,"Targeting Brazil, Trump Tests Legal Limit of H...",https://www.nytimes.com/2025/07/10/us/politics...,33
94,2025-07-10-18-19-52 +0000,bbc,Why is Trump targeting Brazil - and will it ba...,https://www.bbc.com/news/articles/crk68drj57mo,32


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
19,43,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
2,21,2025-07-11-16-33-10 +0000,nypost,Hero Camp Mystic director who died trying to s...,https://nypost.com/2025/07/11/us-news/hero-tex...
45,20,2025-07-11-12-29-49 +0000,nypost,Tom Homan rips violent clashes at California p...,https://nypost.com/2025/07/11/us-news/tom-homa...
66,20,2025-07-11-08-06-08 +0000,bbc,UK-France migrant deal 'robust' against legal ...,https://www.bbc.com/news/articles/cx24d70gw41o
26,19,2025-07-11-14-04-24 +0000,nypost,Sen. Bill Cassidy’s campaign touts record-sett...,https://nypost.com/2025/07/11/us-news/sen-bill...
51,18,2025-07-11-11-21-07 +0000,nypost,Infamous ‘tot mom’ Casey Anthony spotted cozyi...,https://nypost.com/2025/07/11/us-news/casey-an...
56,18,2025-07-11-09-30-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
14,17,2025-07-11-15-41-42 +0000,bbc,Gaza's largest functioning hospital facing dis...,https://www.bbc.com/news/articles/cdx5zeywgrgo
13,17,2025-07-11-15-45-10 +0000,nypost,Hit-and-run BMW driver kills two pedestrians o...,https://nypost.com/2025/07/11/us-news/hit-and-...
48,16,2025-07-11-11-43-52 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
